# Simplex Tableau Calculator

This code was created to help you generate tableaus without working hard.

If you input the matrix A and right hand side b, label the variables, and then choose a basis, it will tell you the tableau at that basis.

Run the cell below that says "Main code - run this cell!"

Then go below and input the data for your problem.  Then you can proceed as the example below for generating the tableaus.

In [23]:
# Main code - run this cell!

import numpy as np

def format_number(x):
    """
    Format the number x as an integer if it is very close to an integer,
    otherwise format with 2 decimal places.
    """
    if abs(x - round(x)) < 1e-10:
        return f"{int(round(x))}"
    else:
        return f"{x:.2f}"

def format_terms(row, nonbasic, var_names):
    """
    Given a tableau row, a list of nonbasic indices, and variable names,
    return a string representing the distributed sum
    \(\sum_{j\in \text{nonbasic}} (-a_{ij})\,x_j\),
    with the negative sign multiplied in.

    In this updated version, even the first term will show a leading + if positive.

    For example, if after distribution the nonbasic coefficients are
      [-1, 2],
    this returns a string like: "- x + 2y".
    """
    terms = []
    for j in nonbasic:
        new_coef = -row[j]  # distribute the negative sign
        if abs(new_coef) < 1e-10:
            continue
        # Omit the number if its absolute value is 1.
        if np.isclose(abs(new_coef), 1.0):
            coef_str = ""
        else:
            coef_str = format_number(abs(new_coef))
        term = f"{coef_str}{var_names[j]}"
        terms.append((new_coef, term))

    expr = ""
    for idx, (coef, term) in enumerate(terms):
        # Always show a leading sign
        sign = "+" if coef >= 0 else "-"
        if idx == 0:
            expr += f"{sign} {term}"
        else:
            expr += f" {sign} {term}"
    return expr

def tableau_to_lp_latex_aligned(tableau, basis_indices, var_names):
    """
    Given a fully pivoted tableau, a basis (list of variable indices),
    and variable names, return a LaTeX string that formats the LP.
    """
    m, n = tableau.shape
    lines = []

    # Identify all variable indices and nonbasic indices
    all_indices = set(range(n-1))
    basic_set = set(basis_indices)
    nonbasic = sorted(list(all_indices - basic_set))

    # --- Objective function ---
    obj_const = format_number(tableau[-1, -1])
    obj_terms = format_terms(tableau[-1, :], nonbasic, var_names)
    obj_expr = obj_terms if obj_const == "0" and obj_terms else f"{obj_const}" + ((" " + obj_terms) if obj_terms else "")
    lines.append(f"\\text{{Maximize}}\\quad & {obj_expr} \\\\")

    # --- Constraints ---
    cons_lines = []
    for i in range(m-1):
        basic_var = var_names[basis_indices[i]]  # Get variable name
        rhs = format_number(tableau[i, -1])
        cons_terms = format_terms(tableau[i, :], nonbasic, var_names)
        expr = f"{rhs}" + ((" " + cons_terms) if cons_terms else "")
        cons_lines.append(f"& {basic_var} = {expr},\\\\ ")

    lines.append("\\text{subject to}\\quad " + cons_lines[0])
    for cl in cons_lines[1:]:
        lines.append(cl)

    all_vars = ", ".join(var_names)
    lines.append(f"& {all_vars} \\ge 0.")

    return (
        "\n"
        "\\begin{aligned}\n"
        + "\n".join(lines) +
        "\n\\end{aligned}\n"
    )


def compute_pivoted_tableau(tableau, basis, var_names):
    """
    Given an initial tableau and a basis list of variable names,
    compute the pivoted tableau corresponding to the provided basis.

    Parameters:
      tableau   : numpy array of shape (m,n)
      basis     : list of variable names (e.g., ["x", "y", "s_1"])
      var_names : list of all variable names in order

    Returns:
      new_tableau : numpy array of shape (m,n) representing the pivoted tableau.
    """
    # Convert basis variable names to their column indices
    basis_indices = [var_names.index(var) for var in basis]

    m, n = tableau.shape
    m_constraints = m - 1
    n_vars = n - 1

    A = tableau[:m_constraints, :n_vars]
    b = tableau[:m_constraints, -1]

    c = tableau[-1, :n_vars]
    z0 = tableau[-1, -1]

    B = A[:, basis_indices]  # Select basic columns
    try:
        B_inv = np.linalg.inv(B)
    except np.linalg.LinAlgError:
        raise ValueError("The chosen basis does not form an invertible matrix.")

    new_A = B_inv @ A
    new_b = B_inv @ b

    c_B = c[basis_indices]
    new_c = c - (np.array(c_B) @ B_inv @ A)

    new_z = z0 - (np.array(c_B) @ B_inv @ b)

    new_tableau = np.zeros_like(tableau)
    new_tableau[:m_constraints, :n_vars] = new_A
    new_tableau[:m_constraints, -1] = new_b
    new_tableau[-1, :n_vars] = new_c
    new_tableau[-1, -1] = new_z

    return new_tableau, basis_indices


from IPython.display import display, Math

def tableau_print(tableau, basis, var_names):
    """
    Print the tableau nicely formatted in Jupyter Notebook, using LaTeX.

    Parameters:
      tableau   : numpy array representing the simplex tableau
      basis     : list of basic variable names (e.g., ["x", "y", "s_1"])
      var_names : list of all variable names in the tableau order
    """
    new_tableau, basis_indices = compute_pivoted_tableau(tableau, basis, var_names)
    latex_str = tableau_to_lp_latex_aligned(new_tableau, basis_indices, var_names)
    display(Math(latex_str))


    # Print BFS and objective value
    solution_dict, obj_value = extract_bfs_and_objective(new_tableau, basis_indices, var_names)

    basic_vars = {var_names[i]: solution_dict[var_names[i]] for i in basis_indices}
    nonbasic_vars = {var_names[i]: solution_dict[var_names[i]] for i in range(len(var_names)) if i not in basis_indices}

    print("\n")
    print("Basic Feasible Solution:")
    print("  Basic Variables:")
    for var, val in basic_vars.items():
        print(f"    {var} = {val:.2f}")
    print("  Nonbasic Variables:")
    for var, val in nonbasic_vars.items():
        print(f"    {var} = {val:.2f}")

    print(f"\nObjective Value: {obj_value:.2f}")



def extract_bfs_and_objective(tableau, basis_indices, var_names):
    """
    Given a final tableau and basis indices, extract the basic feasible solution and objective value.

    Parameters:
      tableau       : final tableau (after pivoting)
      basis_indices : list of indices of basic variables
      var_names     : list of variable names in order

    Returns:
      solution_dict : dictionary of variable values (including zeros for nonbasic)
      obj_value     : float (optimal objective value)
    """
    m, n = tableau.shape
    n_vars = n -1

    solution = np.zeros(n_vars)
    for row_idx, var_idx in enumerate(basis_indices):
        solution[var_idx] = tableau[row_idx, -1]

    obj_value = tableau[-1, -1]

    solution_dict = {var_names[i]: solution[i] for i in range(n_vars)}
    return solution_dict, obj_value


def construct_tableau(A, b, c, z0=0):
    """
    Construct a simplex tableau from separate A, b, c, and z0 inputs.

    Parameters:
      A : 2D numpy array of shape (m, n)
      b : 1D numpy array of shape (m,)
      c : 1D numpy array of shape (n,)
      z0 : scalar (objective constant term)

    Returns:
      tableau : numpy array of shape (m+1, n+1)
    """
    A = np.array(A)
    b = np.array(b).reshape(-1)
    c = np.array(c).reshape(-1)

    m, n = A.shape

    tableau = np.zeros((m+1, n+1))
    tableau[:m, :n] = A
    tableau[:m, -1] = b
    tableau[-1, :n] = -c
    tableau[-1, -1] = z0

    return tableau



### 🧮 Example Linear Program

We are solving a **standard form maximization** linear program with two decision variables $ x $ and $ y $, and three constraints representing resource limits. The slack variables $ s_1, s_2, s_3 $ are introduced to convert inequalities into equalities.

#### 🔧 Decision Variables

- $ x $: amount of good 1 to produce  
- $ y $: amount of good 2 to produce  
- $ s_1, s_2, s_3 $: slack variables for each constraint (represent unused resources)

#### 📈 Objective Function

We aim to **maximize profit**:


$$ \text{Maximize} \quad 2x + 3y $$


####  Constraints

$$
\begin{aligned}
x + y + s_1 &= 9 & \quad \text{(Constraint 1: e.g., labor hours)} \\
2x + y + s_2 &= 16 & \quad \text{(Constraint 2: e.g., flour pounds)} \\
x + 2y + s_3 &= 14 & \quad \text{(Constraint 3: e.g., sugar pounds)}
\end{aligned}
$$

####  Nonnegativity


$$x, y, s_1, s_2, s_3 \geq 0$$


---

This LP models a **resource allocation problem**, such as deciding how many units of two products to produce (say, pies and cakes) given limited resources (labor, flour, sugar), where each product provides a certain profit.


# Input Data Here
Input the data corresponding to the problem, then run the cell to see the dictionary.

In [24]:
# Define A, b, c for a sample LP
A = np.array([
    [1, 1, 1, 0, 0],
    [2, 1, 0, 1, 0],
    [1, 2, 0, 0, 1]
])
b = np.array([9, 16, 14])
c = np.array([2, 3, 0, 0, 0])
z0 = 0

# Construct tableau
tableau = construct_tableau(A, b, c, z0)

# Variable names and chosen initial basis
var_names = ["x", "y", "s_1", "s_2", "s_3"]
basis = ["s_1", "s_2", "s_3"]

# Display LP in LaTeX form
tableau_print(tableau, basis, var_names)


<IPython.core.display.Math object>



Basic Feasible Solution:
  Basic Variables:
    s_1 = 9.00
    s_2 = 16.00
    s_3 = 14.00
  Nonbasic Variables:
    x = 0.00
    y = 0.00

Objective Value: 0.00


Pivot, let $y$ enter and let $s_3$ leave.

In [25]:
basis = ["s_1", "s_2", "y"]

# Display LP in LaTeX form
tableau_print(tableau, basis, var_names)

<IPython.core.display.Math object>



Basic Feasible Solution:
  Basic Variables:
    s_1 = 2.00
    s_2 = 9.00
    y = 7.00
  Nonbasic Variables:
    x = 0.00
    s_3 = 0.00

Objective Value: 21.00


Pivot, let $x$ enter and let $s_1$ leave

In [26]:
basis = ["x", "s_2", "y"]

# Display LP in LaTeX form
tableau_print(tableau, basis, var_names)

<IPython.core.display.Math object>



Basic Feasible Solution:
  Basic Variables:
    x = 4.00
    s_2 = 3.00
    y = 5.00
  Nonbasic Variables:
    s_1 = 0.00
    s_3 = 0.00

Objective Value: 23.00


We reached an optimal solution!

# Infeasible Basis
If we choose a basis that is infeasible, we will see that basic feasible solution has negative values.

In [8]:
basis = ["s_1", "x", "y"]

# Display LP in LaTeX form
tableau_print(tableau, basis, var_names)

<IPython.core.display.Math object>